In [1]:
import pandas as pd
import glob
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar múltiplos arquivos CSV de uma pasta específica
folder_path = 'csvs'  # Substitua pelo caminho da sua pasta
csv_files = glob.glob(folder_path + '/*.csv')

# Definir as colunas a serem importadas e seus tipos de dados
usecols = [
    'hour',
    'total_trip_count',
    'forward_commute_trip_count', 'reverse_commute_trip_count',
    'forward_leisure_trip_count', 'reverse_leisure_trip_count',
    'forward_male_people_count', 'reverse_male_people_count',
    'forward_female_people_count', 'reverse_female_people_count',
    'forward_unspecified_people_count', 'reverse_unspecified_people_count',
    'forward_18_34_people_count', 'reverse_18_34_people_count',
    'forward_35_54_people_count', 'reverse_35_54_people_count',
    'forward_55_64_people_count', 'reverse_55_64_people_count',
    'forward_65_plus_people_count', 'reverse_65_plus_people_count'
]

dtype_dict = {
    'total_trip_count': 'Int16',
    'forward_commute_trip_count': 'Int16', 'reverse_commute_trip_count': 'Int16',
    'forward_leisure_trip_count': 'Int16', 'reverse_leisure_trip_count': 'Int16',
    'forward_male_people_count': 'Int16', 'reverse_male_people_count': 'Int16',
    'forward_female_people_count': 'Int16', 'reverse_female_people_count': 'Int16',
    'forward_unspecified_people_count': 'Int16', 'reverse_unspecified_people_count': 'Int16',
    'forward_18_34_people_count': 'Int16', 'reverse_18_34_people_count': 'Int16',
    'forward_35_54_people_count': 'Int16', 'reverse_35_54_people_count': 'Int16',
    'forward_55_64_people_count': 'Int16', 'reverse_55_64_people_count': 'Int16',
    'forward_65_plus_people_count': 'Int16', 'reverse_65_plus_people_count': 'Int16'
}

# Ler todos os arquivos CSV de uma vez e concatenar em um único DataFrame
df_list = [pd.read_csv(
    file,
    usecols=usecols,
    dtype=dtype_dict,
    parse_dates=['hour']
) for file in csv_files]

df_all = pd.concat(df_list, ignore_index=True)

# Extrair o ano, o mês e o semestre
df_all['year'] = df_all['hour'].dt.year
df_all['month'] = df_all['hour'].dt.month
df_all['semester'] = df_all['month'].apply(lambda x: 1 if x <= 6 else 2)

# Calcular os totais para gêneros e faixas etárias fora da função
df_all['male_people_count'] = df_all['forward_male_people_count'].fillna(0) + df_all['reverse_male_people_count'].fillna(0)
df_all['female_people_count'] = df_all['forward_female_people_count'].fillna(0) + df_all['reverse_female_people_count'].fillna(0)
df_all['unspecified_people_count'] = df_all['forward_unspecified_people_count'].fillna(0) + df_all['reverse_unspecified_people_count'].fillna(0)

df_all['18_34_people_count'] = df_all['forward_18_34_people_count'].fillna(0) + df_all['reverse_18_34_people_count'].fillna(0)
df_all['35_54_people_count'] = df_all['forward_35_54_people_count'].fillna(0) + df_all['reverse_35_54_people_count'].fillna(0)
df_all['55_64_people_count'] = df_all['forward_55_64_people_count'].fillna(0) + df_all['reverse_55_64_people_count'].fillna(0)
df_all['65_plus_people_count'] = df_all['forward_65_plus_people_count'].fillna(0) + df_all['reverse_65_plus_people_count'].fillna(0)

df_all['commute_trip_count'] = df_all['forward_commute_trip_count'].fillna(0) + df_all['reverse_commute_trip_count'].fillna(0)
df_all['leisure_trip_count'] = df_all['forward_leisure_trip_count'].fillna(0) + df_all['reverse_leisure_trip_count'].fillna(0)
df_all['total_trip_count'] = df_all['commute_trip_count'] + df_all['leisure_trip_count']

# Criar widgets conforme solicitado
activity_type_widget = widgets.Dropdown(
    options=['Todos', 'Deslocamento', 'Lazer'],
    value='Todos',
    description='Tipo de Atividade:'
)

ano = widgets.SelectMultiple(
    options=[2019, 2020, 2021, 2022, 2023, 2024],
    value=[2019, 2020, 2021, 2022, 2023, 2024],
    description='Ano:'
)

genero = widgets.Dropdown(
    options=['Todos', 'Homem', 'Mulher', 'Não especificado'],
    value='Todos',
    description='Gênero:'
)

faixa_etaria = widgets.Dropdown(
    options=['Todos', '18-34', '35-54', '55-64', '65+'],
    value='Todos',
    description='Faixa Etária:'
)

# Mapas para mapeamento das opções dos widgets para valores no DataFrame
map_genero = {
    'Todos': 'total',
    'Homem': 'male',
    'Mulher': 'female',
    'Não especificado': 'unspecified'
}

map_atividade = {
    'Todos': 'total',
    'Deslocamento': 'commute',
    'Lazer': 'leisure'
}

map_faixa_etaria = {
    'Todos': 'total',
    '18-34': '18_34',
    '35-54': '35_54',
    '55-64': '55_64',
    '65+': '65_plus'
}

# Definir a função para atualizar o gráfico
def update_plot(activity_type, ano, genero, faixa_etaria):
    # Mapear as opções selecionadas para os valores correspondentes
    genero_mapped = map_genero[genero]
    faixa_etaria_mapped = map_faixa_etaria[faixa_etaria]
    activity_type_mapped = map_atividade[activity_type]

    # Filtrar dados pelos anos selecionados
    df_filtered = df_all[df_all['year'].isin(ano)].copy()

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return

    # Selecionar a coluna de contagem de gênero
    if genero_mapped == 'total':
        df_filtered['people_count'] = df_filtered[['male_people_count', 'female_people_count', 'unspecified_people_count']].sum(axis=1)
    else:
        df_filtered['people_count'] = df_filtered[genero_mapped + '_people_count'].fillna(0)

    # Selecionar a coluna de contagem de faixa etária
    if faixa_etaria_mapped == 'total':
        df_filtered['age_group_count'] = df_filtered[['18_34_people_count', '35_54_people_count', '55_64_people_count', '65_plus_people_count']].sum(axis=1)
    else:
        df_filtered['age_group_count'] = df_filtered[faixa_etaria_mapped + '_people_count'].fillna(0)

    # Filtrar dados onde as contagens de pessoas são maiores que zero
    df_filtered = df_filtered[(df_filtered['people_count'] > 0) & (df_filtered['age_group_count'] > 0)]

    if df_filtered.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return

    # Agrupar os dados por 'year' e 'semester' e somar as contagens de viagens
    grouped = df_filtered.groupby(['year', 'semester']).agg({
        'commute_trip_count': 'sum',
        'leisure_trip_count': 'sum',
        'total_trip_count': 'sum'
    }).reset_index()

    # Transformar o DataFrame para o formato longo adequado para plotagem
    grouped_melted = grouped.melt(
        id_vars=['year', 'semester'],
        value_vars=['total_trip_count', 'commute_trip_count', 'leisure_trip_count'],
        var_name='trip_type',
        value_name='trip_count'
    )

    # Mapear os nomes dos tipos de viagem para português
    trip_type_names = {
        'total_trip_count': 'Total',
        'commute_trip_count': 'Deslocamento',
        'leisure_trip_count': 'Lazer'
    }
    grouped_melted['trip_type'] = grouped_melted['trip_type'].map(trip_type_names)

    # Aplicar o filtro baseado no tipo de atividade selecionado
    if activity_type != 'Todos':
        grouped_melted = grouped_melted[grouped_melted['trip_type'] == activity_type]

    if grouped_melted.empty:
        print("Não há dados disponíveis para os filtros selecionados.")
        return

    # Criar uma coluna combinada de 'year_semester' para o eixo x
    grouped_melted['year_semester'] = grouped_melted['year'].astype(str) + 'S' + grouped_melted['semester'].astype(str)

    # Ordenar os dados para plotagem
    grouped_melted.sort_values(['year', 'semester'], inplace=True)

    # Plotar os dados
    plt.figure(figsize=(14, 7))

    if activity_type == 'Todos':
        # Plotar todos os tipos de atividade com hue='trip_type'
        sns.lineplot(data=grouped_melted, x='year_semester', y='trip_count', hue='trip_type', marker='o')
        plt.legend(title='Tipo de Atividade')
    else:
        # Plotar apenas o tipo de atividade selecionado
        sns.lineplot(data=grouped_melted, x='year_semester', y='trip_count', marker='o')
        plt.legend().set_visible(False)

    plt.title("Viagens por Semestre")
    plt.xlabel('Ano e Semestre')
    plt.ylabel('Número de Viagens')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Tornar a função interativa com os widgets
interact(
    update_plot,
    activity_type=activity_type_widget,
    ano=ano,
    genero=genero,
    faixa_etaria=faixa_etaria
)


interactive(children=(Dropdown(description='Tipo de Atividade:', options=('Todos', 'Deslocamento', 'Lazer'), v…

<function __main__.update_plot(activity_type, ano, genero, faixa_etaria)>